In [10]:
import pandas as pd
import numpy as np
import random
from copy import copy, deepcopy
import torch
import torch.nn as nn
import math

In [74]:
#creating the synthetic data
row_count = 500
n_var = 7

if n_var<=10:
    step_val = 0.1/n_var
else:
    step_val = 0.05/n_var

start_var = np.round(np.random.rand(row_count)*10,0).tolist()
data_dct = {}

for x_var_num in range(n_var):
    data_dct[f'x{x_var_num}'] = []
data_dct[f'x{n_var-1}'] = start_var

all_cols = list(data_dct.keys())

multiply_factors = np.arange(start=1, stop=2, step=step_val)

# an implementation to produce cumulatively increasing array of values row wise
for row_num in range(row_count):
    factor_cpy = copy(multiply_factors)
    for index, x_var_num in enumerate(range(n_var-2,-1,-1)):
        # getting a window of factors as per the x_var_num
        windowed_factors = factor_cpy[index:index+3]
        last_used_factor = np.random.choice(a=windowed_factors,size=1)[0]

        calc_value = np.round(data_dct[f'x{x_var_num+1}'][row_num]*last_used_factor,2)
        
        data_dct[f'x{x_var_num}'].append(calc_value)

        factor_cpy = factor_cpy[factor_cpy>last_used_factor]

In [76]:
main_df = pd.DataFrame(data_dct)
main_df.head(3)

,x0,x1,x2,x3,x4,x5,x6
0,15.35,11.08,8.62,7.27,6.52,6.09,6.0
1,19.95,14.55,11.32,9.43,8.46,8.11,8.0
2,2.18,1.62,1.30,1.12,1.03,1.00,1.0


In [328]:
class PositionalEmbedding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEmbedding, self).__init__()
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model).float()
        pe.require_grad = False

        position = torch.arange(0, max_len).float().unsqueeze(1)
        div_term = (torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model)).exp()

        pe[:, 0::2] = torch.sin(position * div_term)[:pe[:, 0::2].shape[0],:pe[:, 0::2].shape[1]]
        pe[:, 1::2] = torch.cos(position * div_term)[:pe[:, 1::2].shape[0],:pe[:, 1::2].shape[1]]

        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return self.pe[:, :x.size(1)]
    
class TokenEmbedding(nn.Module):
    def __init__(self, c_in, d_model):
        super(TokenEmbedding, self).__init__()
        self.tokenConv = nn.Conv1d(in_channels=c_in, out_channels=d_model, 
                                    kernel_size=3, padding=1)
        # for m in self.modules():
        #     if isinstance(m, nn.Conv1d):
        #         nn.init.kaiming_normal_(m.weight,mode='fan_in',nonlinearity='leaky_relu')

    def forward(self, x):
        # x = self.tokenConv(x.permute(0, 2, 1)).transpose(1,2)
        x = self.tokenConv(x)
        return x

In [324]:

torch.tensor(input, dtype=torch.float32)

tensor([[15.3500, 11.0800,  8.6200,  7.2700,  6.5200,  6.0900,  6.0000]])

In [342]:
input = main_df[all_cols].iloc[:1].to_numpy()#.reshape(1,-1)

tok_emb = TokenEmbedding(c_in=1, d_model=5)
output_tokn = tok_emb(torch.tensor(input, dtype=torch.float32))

print(input.shape)
print(output_tokn.shape)


(1, 7)
torch.Size([5, 7])


In [343]:
output_tokn

tensor([[-2.8626, -5.3464, -4.1868, -3.5255, -3.1587, -2.9843, -1.7063],
        [ 2.2977,  1.5117,  1.4736,  1.4535,  1.4379,  1.4744, -0.2722],
        [-2.1218, -1.7537, -1.4172, -1.2306, -1.1258, -1.0805, -0.5122],
        [-4.1864, -9.3352, -6.8143, -5.3852, -4.6026, -4.1126, -5.8567],
        [ 1.7869, -7.5656, -5.6631, -4.5517, -3.9400, -3.6429, -1.8968]],
       grad_fn=<SqueezeBackward1>)

In [344]:
pos_emb = PositionalEmbedding(d_model=5)
pos_emb(torch.tensor(input))

tensor([[[ 0.0000e+00,  1.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00],
         [ 8.4147e-01,  5.4030e-01,  2.5116e-02,  9.9968e-01,  6.3096e-04],
         [ 9.0930e-01, -4.1615e-01,  5.0217e-02,  9.9874e-01,  1.2619e-03],
         [ 1.4112e-01, -9.8999e-01,  7.5285e-02,  9.9716e-01,  1.8929e-03],
         [-7.5680e-01, -6.5364e-01,  1.0031e-01,  9.9496e-01,  2.5238e-03],
         [-9.5892e-01,  2.8366e-01,  1.2526e-01,  9.9212e-01,  3.1548e-03],
         [-2.7942e-01,  9.6017e-01,  1.5014e-01,  9.8866e-01,  3.7857e-03]]])

In [336]:
output_tokn.T+pos_emb(torch.tensor(input))

tensor([[[-12.0616,  -0.3440,  -2.7621,   4.2544, -10.7779],
         [ -8.1451,   3.2002,   4.4332,  -4.2393,  -1.4906],
         [ -6.4380,   1.7728,   3.0214,  -2.3095,  -1.8753],
         [ -6.3022,   0.9117,   2.2089,  -1.1953,  -2.1326],
         [ -6.6902,   1.0866,   1.7764,  -0.5923,  -2.2617],
         [ -6.6890,   2.0011,   1.5408,  -0.1893,  -2.3993],
         [ -2.5627,   0.1421,   1.7162,  -2.6869,   0.2941]]],
       grad_fn=<AddBackward0>)